In [2]:
import os
from os import mkdir
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xlrd
import random
from matplotlib.collections import LineCollection
from matplotlib.lines import Line2D
import shapely.geometry as geometry
from scipy.spatial import Delaunay
from scipy.spatial import Voronoi, voronoi_plot_2d
from scipy.spatial import distance
from collections import Counter
import geopandas as gpd
from geovoronoi import voronoi_regions_from_coords
import shapely
import math
import matplotlib.tri as tri
from collections import defaultdict
import torch
import time
import multiprocessing as mp

In [3]:
torch.cuda.is_available()

True

In [4]:
def createexcel(path):
    wb = Workbook()
    #with pd.ExcelWriter(path, engine='xlsxwriter') as writer:
     #   print(0)
    wb.save(path)
    print(path, end='\n')
    return newfile

In [5]:
def compute_table(folder):
    count = 0
    inner_path = os.path.join(outer_path, folder)
    total_num_folder = len(folderlist)
    print ('total have %d folders' % (total_num_folder))
    count =count + 1
    print (count)
    filelistn = os.listdir(inner_path)
    path_result = r'/mnt/c/Users/huhui/projects/tiatoolbox/examples/TCGAHCC-result'
    #path0 = r'/mnt/c/Users/huhui/projects/tiatoolbox/examples'
    #path_result0 = os.path.join(path0, "dep_result")
    #path_result = os.path.join(path_result0, str(folder))
    print(path_result, end='\n')
    if not os.path.exists(path_result):
        mkdir(path_result)
    
    for it in range(len(filelistn)):
        #print(folder)
        csvname = filelistn[it]
        #print(csvname[:12])
        src1 = os.path.join(os.path.abspath(inner_path), filelistn[it])
        print(src1, end='\n')
        #inner_patha = os.path.join(inner_path, str(it))
        #print(inner_path)
        #filersult = os.listdir(path)
        #for folder0 in filersult:
           # print(folder0)
        MODEL_DIR = os.path.join(path_result, str(csvname[:12]))#tcga 12 BEIJNG 8
        #print(MODEL_DIR, end='\n')
        if not os.path.exists(MODEL_DIR):
           mkdir(MODEL_DIR)
                
        #log_path = os.path.dirname(os.path.abspath('.')) + '/' + 'data' + '/'
        #path_result = os.path.join(path_result0, str(483970A10))
        #print(path_result, end='\n')
        #log_path = os.path.dirname(path_result) + '/' 
        log_path = MODEL_DIR + '/' 
        print(log_path, end='\n')
        #mkdir(log_path)
        t = str(csvname)
        #suffix = '.xlsx' # 文件类型
        newfile = t 
        pathtt = log_path + newfile
        #start = createexcel(pathtt)
        print(pathtt, end='\n')
        
        
        table = xlrd.open_workbook(src1)
        sheet1 = table.sheet_by_index(0)
        nrow = sheet1.nrows
        print(nrow)
        ncols = sheet1.ncols
        title = sheet1.row_values(1)
        positionx = sheet1.col_values(1)
        positiony = sheet1.col_values(2)
        Category = sheet1.col_values(3)
        minx = min(positionx[1:ncols])
        miny = min(positiony[1:nrow])
        data_listx = []
        data_listy = []
        data_lyx = []
        data_lyy = []
        data_brx = []
        data_bry = []
        data_ca = []
        data_br = []
        data_ly = []
        pointx = []
        pointy = []
        for i in range(1, nrow):
            #x0 = positionx[i] - minx + 1
            #y0 = positiony[i] - miny + 1
            x0 = positionx[i]
            y0 = positiony[i]
            pointx.append(x0)
            pointy.append(y0)
            # data_values = OrderedDict()
            if Category[i] == 3:  # 肿瘤细胞
                data_listx.append(x0)
                data_listy.append(y0)
                data_ca.append((x0, y0))
            if Category[i] == 2:  # 基质细胞
                data_brx.append(x0)
                data_bry.append(y0)
                data_br.append((x0, y0))
            if Category[i] == 1:  # 淋巴细胞
                data_lyx.append(x0)
                data_lyy.append(y0)
                data_ly.append((x0, y0))
        # X=[data_listx, data_listy]
        Point_ca = [data_listx, data_listy]
        num_ca = len(data_ca)
        num_br = len(data_br)
        num_ly = len(data_ly)
        Point_ly = [data_lyx, data_lyy]
        Point_br = [data_brx, data_bry]
        color = []
        ca_ca = 0
        br_br = 0
        ly_ly = 0
        ca_br = 0
        ca_ly = 0
        br_ly = 0
        # computer the distance feature
        dist_cc = []
        dist_bb = []
        dist_bb = []
        dist_ll = []
        dist_cb = []
        dist_cl = []
        dist_bl = []
        all_vertex = []
        all_distance = []
        all_class = []
        vertex_point = []
        X_list = pd.DataFrame({'key': data_listx, 'number': data_listy})
        triangles = tri.Triangulation(pointx, pointy)
        
        SS = triangles.edges

        for i, line in enumerate(SS):
            pointA = (pointx[line[0]], pointy[line[0]])
            pointB = (pointx[line[1]], pointy[line[1]])
            dist = distance.euclidean(pointA, pointB)
            all_vertex.append(line[0])
            vertex_point.append([pointx[line[0]], pointy[line[0]]])
            all_vertex.append(line[1])
            vertex_point.append([pointx[line[1]], pointy[line[1]]])
            all_distance.append(dist)
            all_distance.append(dist)
            if pointA in data_ca and pointB in data_ca:
                color.append(0)
                color.append(0)
                all_class.append(3)
                all_class.append(3)
                ca_ca += 1
            elif pointA in data_ly and pointB in data_ly:
                color.append(50)
                color.append(50)
                all_class.append(1)
                all_class.append(1)
                ly_ly += 1
            elif pointA in data_br and pointB in data_br:
                color.append(100)
                color.append(100)
                all_class.append(2)
                all_class.append(2)
                br_br += 1
            elif pointA in data_ca and pointB in data_br:
                color.append(150)
                color.append(150)
                all_class.append(3)
                all_class.append(2)
                ca_br += 1
            elif pointA in data_br and pointB in data_ca:
                color.append(150)
                color.append(150)
                all_class.append(2)
                all_class.append(3)
                ca_br += 1
            elif pointA in data_br and pointB in data_ly:
                color.append(200)
                color.append(200)
                all_class.append(2)
                all_class.append(1)
                br_ly += 1
            elif pointA in data_ly and pointB in data_br:
                color.append(200)
                color.append(200)
                all_class.append(1)
                all_class.append(2)
                br_ly += 1
            elif pointA in data_ca and pointB in data_ly:
                color.append(255)
                color.append(255)
                all_class.append(3)
                all_class.append(1)
                ca_ly += 1
            elif pointA in data_ly and pointB in data_ca:
                color.append(255)
                color.append(255)
                all_class.append(1)
                all_class.append(3)
                ca_ly += 1


        all_edge = len(SS)
        #总连接边的总体距离的2倍
        total_distance = sum(all_distance)/2
        #总连接边的总体数量的2倍
        total_edge = len(all_distance)/2

        name_attribute = ['total_distance', 'total_edge', 'num_ca', 'num_br', 'num_ly']
        #dataini = pd.DataFrame(columns=('numberx', 'numbery'))
        writerCSVe = pd.DataFrame(columns=name_attribute)
        dataedge = {'total_distance': total_distance, 'total_edge': total_edge, 'num_ca': num_ca, 'num_br': num_br, 'num_ly': num_ly}
        datae =writerCSVe._append(pd.DataFrame(dataedge, index=[0]))
        #datae.to_csv(dataedge)
        #filee.close()
        all_cluster = sum(all_distance) / len(all_distance)
        name_attribute = ['all_vertex', 'vertex_point',  'all_distance',  'all_class', 'color']
        writerCSV1 = pd.DataFrame(columns=name_attribute)
        data1 = {'all_vertex': all_vertex, 'vertex_point': vertex_point, 'all_distance': all_distance, 'all_class': all_class, 'color': color}
        datatri =writerCSV1._append(pd.DataFrame(data1))

        #可以在支部程序中计算
        dict_distance = defaultdict(list)
        dict_class = defaultdict(list)
        dict_color = defaultdict(list)
        for item, distancea in zip(color, all_distance):
            dict_color[item].append(distancea)

        for vertex, distancea in zip(all_vertex, all_distance):
            dict_distance[vertex].append(distancea)
        for vertex, distancea in zip(all_vertex, all_distance):
            dict_distance[vertex].append(distancea)
        for vertex, classnum in zip(all_vertex, all_class):
            dict_class[vertex].append(classnum)


        # create Tayor neterok
        X = [pointx, pointy]
        X = np.array(np.transpose(X))
        vor = Voronoi(X)
        voronoi_vertices = vor.vertices
        #voronoi_regions = vor.regions
        voronoi_regions = list(filter(None, vor.regions))
        voronoi_points = list(vor.points)
        list_point =list(vor.point_region)
        inner_point = []
        for i, region in enumerate(voronoi_regions):
            if -1 not in region:
                #if len(region)>0:
                polygon = [voronoi_vertices[p] for p in region]
                if i in list_point:
                   ss = list_point.index(i)
                   inner_point.append(vor.points[ss])
        #diff_list = list(set(voronoi_points) - set(inner_point))
        #print(diff_list)
        #a = [x for x in voronoi_points if x in inner_point]
        list_pointv = []
        list_inner = []
        for point in inner_point:
            pointv = (point[0], point[1])
            list_inner.append(pointv)
        for point in voronoi_points:
            pointv = (point[0], point[1])
            list_pointv.append(pointv)
        outer_point = [x for x in list_pointv if x not in list_inner]
        #print(outer_point)

        #reconstruction voronoi and remove outlier cells
        linesa = [shapely.geometry.LineString(vor.vertices[line]) for line in
                 vor.ridge_vertices if -1 not in line]
        #lines = [shapely.geometry.LineString(vor.vertices[line]) for line in
                 #vor.ridge_vertices if -1 not in line]
        #lines2 = [shapely.geometry.LineString(vor.vertices[line]) for line in
                 #vor.ridge_vertices if -1 in line]
        polys = shapely.ops.polygonize(linesa)
        voronois = gpd.GeoDataFrame(geometry=gpd.GeoSeries(polys))
        voro_area = voronois.T.T.area
        ca_area = []
        ly_area = []
        br_area = []
        type_area = []
        vor_area = []

        for i, point in enumerate(inner_point):
            pointv = (point[0], point[1])
            vor_area.append(voro_area[i])
            if pointv in data_ca:
                ca_area.append(voro_area[i])
                type_area.append(3)
            if pointv in data_br:
                br_area.append(voro_area[i])
                type_area.append(2)
            if pointv in data_ly:
                ly_area.append(voro_area[i])
                type_area.append(1)
        name_attribute = ['NumberID',  'vor_area',  'type_area']
        #dataini = pd.DataFrame(columns=('numberx', 'numbery'))
        writerCSV = pd.DataFrame(columns=name_attribute)
        data = {'NumberID': list_inner, 'vor_area': vor_area, 'type_area': type_area}
        dataini =writerCSV._append(pd.DataFrame(data))

        # 声明一个读写对象
        # with pd.ExcelWriter(r'datanew/TCGA-2Y-A9GW-01Z-00-DX1.71805205-933D-4D72-A4A2-586DC5490D76_1_Info.xlsx', engine='xlsxwriter') as writer:
        with pd.ExcelWriter(pathtt, engine='xlsxwriter') as writer:
            datatri.to_excel(writer, sheet_name='tri')
            dataini.to_excel(writer, sheet_name='vor')
            datae.to_excel(writer, sheet_name='edge')
        
        
        
        

In [7]:
if __name__ == '__main__':
    #table = xlrd.open_workbook(r'432831A4_7_Info.xlsx')
    start = time.time()
    outer_path = '/mnt/c/Users/huhui/projects/tiatoolbox/examples/HCC-all0'
    # Root directory of the project
    ROOT_DIR = os.getcwd()
    print(ROOT_DIR)
    it=1
    folderlist = os.listdir(outer_path)
    for folder in folderlist:
        results = compute_table(folder)  
        #writer.writerow(results)
    #for processedimg in results:
      #  cv2.imwrite("result/"+str(i)+'.jpg', processedimg)
      #  i+=1
    print(time.time()-start)
    

/mnt/c/Users/huhui/projects/tiatoolbox/examples
total have 1 folders
1
/mnt/c/Users/huhui/projects/tiatoolbox/examples/TCGAHCC-result
/mnt/c/Users/huhui/projects/tiatoolbox/examples/HCC-all0/1/TCGA-2Y-A9GT-01Z-00-DX1.30666775-3556-4DFE-A5EC-8CCF8EEB1803_10_Info.xlsx
/mnt/c/Users/huhui/projects/tiatoolbox/examples/TCGAHCC-result/TCGA-2Y-A9GT/
/mnt/c/Users/huhui/projects/tiatoolbox/examples/TCGAHCC-result/TCGA-2Y-A9GT/TCGA-2Y-A9GT-01Z-00-DX1.30666775-3556-4DFE-A5EC-8CCF8EEB1803_10_Info.xlsx
5287
/mnt/c/Users/huhui/projects/tiatoolbox/examples/HCC-all0/1/TCGA-2Y-A9GT-01Z-00-DX1.30666775-3556-4DFE-A5EC-8CCF8EEB1803_11_Info.xlsx
/mnt/c/Users/huhui/projects/tiatoolbox/examples/TCGAHCC-result/TCGA-2Y-A9GT/
/mnt/c/Users/huhui/projects/tiatoolbox/examples/TCGAHCC-result/TCGA-2Y-A9GT/TCGA-2Y-A9GT-01Z-00-DX1.30666775-3556-4DFE-A5EC-8CCF8EEB1803_11_Info.xlsx
5199
/mnt/c/Users/huhui/projects/tiatoolbox/examples/HCC-all0/1/TCGA-2Y-A9GT-01Z-00-DX1.30666775-3556-4DFE-A5EC-8CCF8EEB1803_12_Info.xlsx
/mn

In [26]:
#no use
def compute_table(folder):
    count = 0
    inner_path = os.path.join(outer_path, folder)
    total_num_folder = len(folderlist)
    print ('total have %d folders' % (total_num_folder))
    count =count + 1
    print (count)
    filelistn = os.listdir(inner_path)
    path_result0 = r'/mnt/c/Users/huhui/projects/tiatoolbox/examples/deptest-result'
    #path0 = r'/mnt/c/Users/huhui/projects/tiatoolbox/examples'
    #path_result0 = os.path.join(path0, "dep_result")
    path_result = os.path.join(path_result0, str(folder))
    print(path_result, end='\n')
    if not os.path.exists(path_result):
        mkdir(path_result)
    
    for it in range(len(filelistn)):
        #print(folder)
        csvname = filelistn[it]
        print(len(csvname))
        src1 = os.path.join(os.path.abspath(inner_path), filelistn[it])
        #print(src1, end='\n')
        #inner_patha = os.path.join(inner_path, str(it))
        #print(inner_path)
        #filersult = os.listdir(path)
        #for folder0 in filersult:
           # print(folder0)
        MODEL_DIR = os.path.join(path_result, str(csvname))
        #print(MODEL_DIR, end='\n')
        #if not os.path.exists(MODEL_DIR):
           #mkdir(MODEL_DIR)
        
        
        table = xlrd.open_workbook(src1)
        sheet1 = table.sheet_by_index(0)
        nrow = sheet1.nrows
        print(nrow)
        ncols = sheet1.ncols
        title = sheet1.row_values(1)
        positionx = sheet1.col_values(1)
        positiony = sheet1.col_values(2)
        Category = sheet1.col_values(3)
        minx = min(positionx[1:ncols])
        miny = min(positiony[1:nrow])
        data_listx = []
        data_listy = []
        data_lyx = []
        data_lyy = []
        data_brx = []
        data_bry = []
        data_ca = []
        data_br = []
        data_ly = []
        pointx = []
        pointy = []
        for i in range(1, nrow):
            #x0 = positionx[i] - minx + 1
            #y0 = positiony[i] - miny + 1
            x0 = positionx[i]
            y0 = positiony[i]
            pointx.append(x0)
            pointy.append(y0)
            # data_values = OrderedDict()
            if Category[i] == 3:  # 肿瘤细胞
                data_listx.append(x0)
                data_listy.append(y0)
                data_ca.append((x0, y0))
            if Category[i] == 2:  # 基质细胞
                data_brx.append(x0)
                data_bry.append(y0)
                data_br.append((x0, y0))
            if Category[i] == 1:  # 淋巴细胞
                data_lyx.append(x0)
                data_lyy.append(y0)
                data_ly.append((x0, y0))
        # X=[data_listx, data_listy]
        Point_ca = [data_listx, data_listy]
        num_ca = len(data_ca)
        num_br = len(data_br)
        num_ly = len(data_ly)
        Point_ly = [data_lyx, data_lyy]
        Point_br = [data_brx, data_bry]
        color = []
        ca_ca = 0
        br_br = 0
        ly_ly = 0
        ca_br = 0
        ca_ly = 0
        br_ly = 0
        # computer the distance feature
        dist_cc = []
        dist_bb = []
        dist_bb = []
        dist_ll = []
        dist_cb = []
        dist_cl = []
        dist_bl = []
        all_vertex = []
        all_distance = []
        all_class = []
        vertex_point = []
        X_list = pd.DataFrame({'key': data_listx, 'number': data_listy})
        triangles = tri.Triangulation(pointx, pointy)
        
        SS = triangles.edges

        for i, line in enumerate(SS):
            pointA = (pointx[line[0]], pointy[line[0]])
            pointB = (pointx[line[1]], pointy[line[1]])
            dist = distance.euclidean(pointA, pointB)
            all_vertex.append(line[0])
            vertex_point.append([pointx[line[0]], pointy[line[0]]])
            all_vertex.append(line[1])
            vertex_point.append([pointx[line[1]], pointy[line[1]]])
            all_distance.append(dist)
            all_distance.append(dist)
            if pointA in data_ca and pointB in data_ca:
                color.append(0)
                color.append(0)
                all_class.append(3)
                all_class.append(3)
                ca_ca += 1
            elif pointA in data_ly and pointB in data_ly:
                color.append(50)
                color.append(50)
                all_class.append(1)
                all_class.append(1)
                ly_ly += 1
            elif pointA in data_br and pointB in data_br:
                color.append(100)
                color.append(100)
                all_class.append(2)
                all_class.append(2)
                br_br += 1
            elif pointA in data_ca and pointB in data_br:
                color.append(150)
                color.append(150)
                all_class.append(3)
                all_class.append(2)
                ca_br += 1
            elif pointA in data_br and pointB in data_ca:
                color.append(150)
                color.append(150)
                all_class.append(2)
                all_class.append(3)
                ca_br += 1
            elif pointA in data_br and pointB in data_ly:
                color.append(200)
                color.append(200)
                all_class.append(2)
                all_class.append(1)
                br_ly += 1
            elif pointA in data_ly and pointB in data_br:
                color.append(200)
                color.append(200)
                all_class.append(1)
                all_class.append(2)
                br_ly += 1
            elif pointA in data_ca and pointB in data_ly:
                color.append(255)
                color.append(255)
                all_class.append(3)
                all_class.append(1)
                ca_ly += 1
            elif pointA in data_ly and pointB in data_ca:
                color.append(255)
                color.append(255)
                all_class.append(1)
                all_class.append(3)
                ca_ly += 1


        all_edge = len(SS)
        #总连接边的总体距离的2倍
        total_distance = sum(all_distance)/2
        #总连接边的总体数量的2倍
        total_edge = len(all_distance)/2

        name_attribute = ['total_distance', 'total_edge']
        #dataini = pd.DataFrame(columns=('numberx', 'numbery'))
        writerCSVe = pd.DataFrame(columns=name_attribute)
        dataedge = {'total_distance': total_distance, 'total_edge': total_edge}
        datae =writerCSVe._append(pd.DataFrame(dataedge, index=[0]))
        #datae.to_csv(dataedge)
        #filee.close()
        all_cluster = sum(all_distance) / len(all_distance)
        name_attribute = ['all_vertex', 'vertex_point',  'all_distance',  'all_class', 'color']
        writerCSV1 = pd.DataFrame(columns=name_attribute)
        data1 = {'all_vertex': all_vertex, 'vertex_point': vertex_point, 'all_distance': all_distance, 'all_class': all_class, 'color': color}
        datatri =writerCSV1._append(pd.DataFrame(data1))

        #可以在支部程序中计算
        dict_distance = defaultdict(list)
        dict_class = defaultdict(list)
        dict_color = defaultdict(list)
        for item, distancea in zip(color, all_distance):
            dict_color[item].append(distancea)

        for vertex, distancea in zip(all_vertex, all_distance):
            dict_distance[vertex].append(distancea)
        for vertex, distancea in zip(all_vertex, all_distance):
            dict_distance[vertex].append(distancea)
        for vertex, classnum in zip(all_vertex, all_class):
            dict_class[vertex].append(classnum)


        # create Tayor neterok
        X = [pointx, pointy]
        X = np.array(np.transpose(X))
        vor = Voronoi(X)
        voronoi_vertices = vor.vertices
        #voronoi_regions = vor.regions
        voronoi_regions = list(filter(None, vor.regions))
        voronoi_points = list(vor.points)
        list_point =list(vor.point_region)
        inner_point = []
        for i, region in enumerate(voronoi_regions):
            if -1 not in region:
                #if len(region)>0:
                polygon = [voronoi_vertices[p] for p in region]
                if i in list_point:
                   ss = list_point.index(i)
                   inner_point.append(vor.points[ss])
        #diff_list = list(set(voronoi_points) - set(inner_point))
        #print(diff_list)
        #a = [x for x in voronoi_points if x in inner_point]
        list_pointv = []
        list_inner = []
        for point in inner_point:
            pointv = (point[0], point[1])
            list_inner.append(pointv)
        for point in voronoi_points:
            pointv = (point[0], point[1])
            list_pointv.append(pointv)
        outer_point = [x for x in list_pointv if x not in list_inner]
        #print(outer_point)

        #reconstruction voronoi and remove outlier cells
        linesa = [shapely.geometry.LineString(vor.vertices[line]) for line in
                 vor.ridge_vertices if -1 not in line]
        #lines = [shapely.geometry.LineString(vor.vertices[line]) for line in
                 #vor.ridge_vertices if -1 not in line]
        #lines2 = [shapely.geometry.LineString(vor.vertices[line]) for line in
                 #vor.ridge_vertices if -1 in line]
        polys = shapely.ops.polygonize(linesa)
        voronois = gpd.GeoDataFrame(geometry=gpd.GeoSeries(polys))
        voro_area = voronois.T.T.area
        ca_area = []
        ly_area = []
        br_area = []
        type_area = []
        vor_area = []

        for i, point in enumerate(inner_point):
            pointv = (point[0], point[1])
            vor_area.append(voro_area[i])
            if pointv in data_ca:
                ca_area.append(voro_area[i])
                type_area.append(3)
            if pointv in data_br:
                br_area.append(voro_area[i])
                type_area.append(2)
            if pointv in data_ly:
                ly_area.append(voro_area[i])
                type_area.append(1)
        name_attribute = ['NumberID',  'vor_area',  'type_area']
        #dataini = pd.DataFrame(columns=('numberx', 'numbery'))
        writerCSV = pd.DataFrame(columns=name_attribute)
        data = {'NumberID': list_inner, 'vor_area': vor_area, 'type_area': type_area}
        dataini =writerCSV._append(pd.DataFrame(data))
    
    
        #log_path = os.path.dirname(os.path.abspath('.')) + '/' + 'data' + '/'
        #path_result = os.path.join(path_result0, str(483970A10))
        #print(path_result, end='\n')
        #log_path = os.path.dirname(path_result) + '/' 
        log_path = os.path.dirname(path_result) + '/' 
        print(log_path, end='\n')
        #mkdir(log_path)
        t = str(csvname)
        #suffix = '.xlsx' # 文件类型
        newfile = t 
        pathtt = log_path + newfile
        #start = createexcel(pathtt)
        print(pathtt, end='\n')
        # 声明一个读写对象
        # with pd.ExcelWriter(r'datanew/TCGA-2Y-A9GW-01Z-00-DX1.71805205-933D-4D72-A4A2-586DC5490D76_1_Info.xlsx', engine='xlsxwriter') as writer:
        with pd.ExcelWriter(pathtt, engine='xlsxwriter') as writer:
            datatri.to_excel(writer, sheet_name='tri')
            dataini.to_excel(writer, sheet_name='vor')
            datae.to_excel(writer, sheet_name='edge')

In [29]:
#no use
if __name__ == '__main__':
    #table = xlrd.open_workbook(r'432831A4_7_Info.xlsx')
    start = time.time()
    outer_path = '/mnt/c/Users/huhui/projects/tiatoolbox/examples/deptest'
    # Root directory of the project
    ROOT_DIR = os.getcwd()
    print(ROOT_DIR)
    it=1
    folderlist = os.listdir(outer_path)
    for folder in folderlist:
        results = compute_table(folder)  
        #writer.writerow(results)
    #for processedimg in results:
      #  cv2.imwrite("result/"+str(i)+'.jpg', processedimg)
      #  i+=1
    print(time.time()-start)

/mnt/c/Users/huhui/projects/tiatoolbox/examples
total have 1 folders
1
/mnt/c/Users/huhui/projects/tiatoolbox/examples/deptest-result/1
72
72
72
0.013195514678955078


In [ ]:
#no use
if __name__ == '__main__':
    #table = xlrd.open_workbook(r'432831A4_7_Info.xlsx')
    start = time.time()
    outer_path = '/mnt/c/Users/huhui/projects/tiatoolbox/examples/depnew26'
    # Root directory of the project
    ROOT_DIR = os.getcwd()
    print(ROOT_DIR)
    it=1
    folderlist = os.listdir(outer_path)
    pool = mp.Pool(mp.cpu_count())
    #results = pool.map(compute_table, [folder for folder in folderlist])
    results = pool.map(compute_table, [folder for folder in folderlist])
    #for processedimg in results:
      #  cv2.imwrite("result/"+str(i)+'.jpg', processedimg)
      #  i+=1
    pool.close()
    print(time.time()-start)